<a href="https://colab.research.google.com/github/Sohammhatre10/Predicto/blob/main/NewsAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.2 MB/s eta 0:00:00


In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at human-centered-summarization/financial-summarization-pegasus and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:

from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [4]:
monitored_ticker = ['Bitcoin', 'Dogecoin', 'Litecoin', 'Ethereum', 'XRP']

In [5]:
def search_for_crypto_news(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&sca_esv=554867176&rlz=1C1ONGR_enIN1063IN1063&tbs=sbd:1,qdr:d&tbm=nws&sxsrf=AB5stBjN5-syjYtVHcPU9P30_SJyTV9fUg:1691523802642&source=lnt&sa=X&ved=2ahUKEwi7s4qb6c2AAxUmpVYBHfCNBSoQpwV6BAgBEAg&biw=1280&bih=595&dpr=1.5".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [6]:
raw_urls = {}
for ticker in monitored_ticker:
  raw_urls[ticker] = search_for_crypto_news(ticker)

In [7]:
import re

In [8]:
exclude_list = ['maps', 'policies','preferences', 'accounts', 'support']

In [9]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [10]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_ticker}
cleaned_urls

{'Bitcoin': ['https://finance.yahoo.com/news/virtune-ab-publ-launching-virtune-062900090.html',
  'https://finance.yahoo.com/news/coinbase-exec-no-playbook-public-013122860.html',
  'https://finance.yahoo.com/news/bitcoin-flashes-signals-possible-spike-072044372.html',
  'https://finance.yahoo.com/news/bitcoins-taker-buy-sell-ratio-072614200.html',
  'https://finance.yahoo.com/news/fed-looks-increase-oversight-banks-023922850.html',
  'https://finance.yahoo.com/news/bitcoin-ether-rise-near-key-095010837.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BBitcoin%26sca_esv%3D554867176%26rlz%3D1C1ONGR_enIN1063IN1063%26tbs%3Dsbd:1,qdr:d%26tbm%3Dnws%26sxsrf%3DAB5stBjN5-syjYtVHcPU9P30_SJyTV9fUg:1691523802642%26source%3Dlnt%26sa%3DX%26ved%3D2ahUKEwi7s4qb6c2AAxUmpVYBHfCNBSoQpwV6BAgBEAg%26biw%3D1280%26bih%3D595%26dpr%3D1.5%26pccc%3D1',
  'https://finance.yahoo.com/news/common-scenarios-lost-crypto-cybernetics-054012377.html',
  'https://finance.yahoo.com/news/ark-invest-bitcoin-etf-e

In [14]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [13]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_ticker}
articles

{'Bitcoin': ["Stockholm, August 9th, 2023 - Virtune, a Swedish regulated digital asset manager in Sweden is launching a new innovative staking ETP on Nasdaq Stockholm, the largest stock exchange in the Nordic region. Today, Virtune is announcing the introduction of Virtune Staked Ethereum ETP SEK, providing exposure to Ethereum combined with the benefits of staking.  The ETP which is 100% physically backed and fully collateralized is denominated in SEK for the Swedish audience and is available on Avanza and Nordnet. By including staking, this ETP provides investors with up to 5% in additional annual return on their ETP Ethereum investment.  Ethereum is currently the second largest cryptocurrency with a market cap of about $219 billion and the world's leading smart contract platform which can revolutionize the way we interact with technology and finance. With its robust ecosystem, Ethereum enables developers to build decentralized applications (dApps) that can reshape industries, elimin

In [15]:
btc = articles["Bitcoin"]
eth = articles["Ethereum"]
lte = articles["Litecoin"]
xrp = articles["XRP"]
doge = articles["Dogecoin"]

In [16]:
def summarize(articles):
    summaries = []
    for ticker, article_list in articles.items():
        article_text = '\n\n'.join(article_list)  # Join the list of articles into a single string
        ticker_summaries = []
        for chunk in article_text.split('\n\n'):  # Split the article into paragraphs (you can adjust this based on your article structure)
            input_ids = tokenizer.encode(chunk, return_tensors='pt', max_length=512, truncation=True)
            output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
            summary = tokenizer.decode(output[0], skip_special_tokens=True)
            ticker_summaries.append(summary)
        summaries.append({ticker: ticker_summaries})
    return summaries

# Assuming you have a dictionary with lists of articles for each ticker (e.g., btc, eth, doge, xrp, lte)
articles = {
    'Bitcoin': btc,
    'Ethereum': eth,
    'Dogecoin': doge,
    'XRP': xrp,
    'Litecoin': lte
}

# Call the summarize function with the articles dictionary
summaries = summarize(articles)
print(summaries)

[{'Bitcoin': ['Virtune to launch a new innovative staking ETP on Nasdaq Stockholm.', '‘There’s no playbook,’ says Jesse Pollak, creator of Base. Exchange preparing for the official launch of its new blockchain', 'Chart patterns suggest Bitcoin may be approaching lows. Bitcoin’s 2023 rebound has cooled amid low volatility', 'Hit a three-and-a-half-month high on BitMEX, OKX on Tuesday. Signs of renewed bullish sentiment seen in taker buy-sell ratio', 'New program aims to ‘supervise novel activities’ in banks. Agency’s stance on stablecoins seen as bearish for industry', 'Coin little changed in Asia after falling below US$30,000 support level. Solana, Polygon, DraftKings among day’s top 10 winners', 'All images are copyrighted.', 'Cybernetics introduces cutting-edge solutions to mitigate risks associated with lost crypto assets.', 'Wood expects more than one Bitcoin ETF to be approved. ARK Invest and 21Shares Bitcoin ETF are in lead among applications.', 'Bitcoin volatility continues to b

In [20]:
from transformers import pipeline
import pandas as pd


# Initialize the sentiment analysis pipeline
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_revision = "af0f99b"
sentiment_classifier = pipeline('sentiment-analysis', model=model_name, revision=model_revision)

def analyze_sentiment(summaries):
    sentiment_results = sentiment_classifier(summaries)  # Use the pipeline directly on the list of summaries
    return sentiment_results

# Assuming you want to analyze the sentiment for the 'BTC' ticker's summaries
btc_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('BTC', [])]
eth_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('ETH', [])]  # List of summaries for ETH
doge_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('DOGE', [])]  # List of summaries for DOGE
xrp_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('XRP', [])]  # List of summaries for XRP
lte_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('LTE', [])]  # List of summaries for LTE

sentiment_results_btc = analyze_sentiment(btc_summaries)
sentiment_results_eth = analyze_sentiment(eth_summaries)
sentiment_results_doge = analyze_sentiment(doge_summaries)
sentiment_results_xrp = analyze_sentiment(xrp_summaries)
sentiment_results_lte = analyze_sentiment(lte_summaries)


data = {
    'Ticker': ['BTC'] * len(sentiment_results_btc) + ['ETH'] * len(sentiment_results_eth) +
              ['DOGE'] * len(sentiment_results_doge) + ['XRP'] * len(sentiment_results_xrp) +
              ['LTE'] * len(sentiment_results_lte),
    'Result_state': [result['label'] for result in sentiment_results_btc] +
                       [result['label'] for result in sentiment_results_eth] +
                       [result['label'] for result in sentiment_results_doge] +
                       [result['label'] for result in sentiment_results_xrp] +
                       [result['label'] for result in sentiment_results_lte],
    'Score': [result['score'] for result in sentiment_results_btc] +
                       [result['score'] for result in sentiment_results_eth] +
                       [result['score'] for result in sentiment_results_doge] +
                       [result['score'] for result in sentiment_results_xrp] +
                       [result['score'] for result in sentiment_results_lte],
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('sentiment_analysis_results.csv', index=False)

In [49]:
import math

coin_results = {}
coins = ['Bitcoin', 'Ethereum', 'XRP', 'Litecoin', 'Dogecoin']

for coin in coins:
    positive_sum = df[(df['Ticker'] == coin) & (df['Result_state'] == 'POSITIVE')]['Score'].sum()
    negative_sum = df[(df['Ticker'] == coin) & (df['Result_state'] == 'NEGATIVE')]['Score'].sum()
    coin_results[coin] = 1+(((positive_sum - negative_sum)/(pow(len(df[(df['Ticker'] == coin)]),(13589029/10000000)))))
print(coin_results)





{'Bitcoin': nan, 'Ethereum': nan, 'XRP': 1.0464872529938258, 'Litecoin': nan, 'Dogecoin': nan}


<ipython-input-49-9bb52695ef0c>:9: RuntimeWarning: invalid value encountered in double_scalars
  coin_results[coin] = 1+(((positive_sum - negative_sum)/(pow(len(df[(df['Ticker'] == coin)]),(13589029/10000000)))))
